In [3]:
import pandas as pd

def find_status_entries(csv_path, keyword):
    try:
        # 1. 读取CSV文件
        df = pd.read_csv(csv_path)
        
        # 2. 检查是否存在"status"列
        if "status" not in df.columns:
            print("错误：CSV文件中没有名为'status'的列。")
            return
        
        # 3. 查找包含关键词的行（不区分大小写）
        mask = df["status"].str.contains(keyword, case=False, na=False)
        matched_rows = df[mask]
        
        # 4. 输出结果
        if not matched_rows.empty:
            print(f"找到 {len(matched_rows)} 条包含'{keyword}'的记录：")
            for index, row in matched_rows.iterrows():
                print("\n位置（行索引）:", index)
                print("数据内容:")
                print(row.to_string())
        else:
            print(f"未找到包含'{keyword}'的记录。")
            
    except FileNotFoundError:
        print(f"错误：文件 '{csv_path}' 未找到。")
    except Exception as e:
        print(f"发生未知错误: {str(e)}")

# 使用示例
if __name__ == "__main__":
    csv_file = r"D:\竞赛\数据挖掘竞赛\其他参赛者的答案\baseline解答\DeepSeaGLM-main\DeepSeaGLM-main\baseline\mountain_baseline\database_in_use\Ajia_plc_1.csv"  # 替换为你的CSV文件路径
    target_keyword = "作业"  # 要查找的关键词
    find_status_entries(csv_file, target_keyword)

未找到包含'作业'的记录。


In [14]:
import os
import sys
import argparse

def validate_directory(path):
    """深度验证目录有效性"""
    if not os.path.exists(path):
        raise NotADirectoryError(f"路径不存在: {path}")
    if not os.path.isdir(path):
        raise NotADirectoryError(f"不是有效目录: {path}")
    if not os.access(path, os.R_OK):
        raise PermissionError(f"无读取权限: {path}")
    return os.path.abspath(path)

def get_target_dir():
    """多途径获取目标目录"""
    # # 方式1：命令行参数
    # parser = argparse.ArgumentParser(description='合并指定目录的代码文件')
    # parser.add_argument('-d', '--directory', type=str, help='目标目录路径')
    # args = parser.parse_args()
    # 
    # if args.directory:
    #     return validate_directory(args.directory)
    # 
    # # 方式2：拖放目录到脚本图标
    # if len(sys.argv) > 1:
    #     drag_path = sys.argv[1]
    #     try:
    #         return validate_directory(drag_path)
    #     except Exception as e:
    #         print(f"拖放路径无效: {e}")
    
    # 方式3：交互式输入
    while True:
        input_path = input("请输入目标目录的完整路径（或拖放目录到这里）: ").strip('"')
        try:
            return validate_directory(input_path)
        except Exception as e:
            print(f"无效路径: {e}\n请重新输入")

def merge_files(target_dir):
    output_path = os.path.join(target_dir, "merged_files.txt")
    
    # 收集目标文件（保持大小写敏感）
    files = []
    for entry in os.scandir(target_dir):
        if entry.is_file() and entry.name.lower().endswith(('.py', '.json')):
            files.append(entry.path)
    
    # 智能排序：先按扩展名分类，再按文件名排序
    files.sort(key=lambda x: (os.path.splitext(x)[1].lower(), os.path.basename(x).lower()))
    
    with open(output_path, 'w', encoding='utf-8') as output:
        for file_path in files:
            # 写入带样式的文件头
            output.write(f"\n{'#' * 80}\n### 文件路径: {file_path}\n{'#' * 80}\n\n")
            
            try:
                # 自动检测编码
                with open(file_path, 'rb') as f:
                    raw_data = f.read()
                    try:
                        content = raw_data.decode('utf-8')
                    except UnicodeDecodeError:
                        content = raw_data.decode('gbk', errors='replace')
                
                # 统一换行符并移除末尾多余空行
                content = content.replace('\r\n', '\n').rstrip()
                output.write(f"{content}\n")
                
            except Exception as e:
                error_msg = f"【读取失败】{type(e).__name__}: {str(e)}"
                output.write(f"{error_msg}\n")
    
    print(f"合并完成！输出文件路径: {os.path.abspath(output_path)}")

if __name__ == '__main__':
    try:
        target_dir = get_target_dir()
        print(f"正在处理目录: {target_dir}")
        merge_files(target_dir)
    except KeyboardInterrupt:
        print("\n操作已取消")
    except Exception as e:
        print(f"严重错误: {str(e)}", file=sys.stderr)
        sys.exit(1)

正在处理目录: D:\竞赛\数据挖掘竞赛\其他参赛者的答案\baseline解答\DeepSeaGLM-main\DeepSeaGLM-main\baseline\mountain_baseline
合并完成！输出文件路径: D:\竞赛\数据挖掘竞赛\其他参赛者的答案\baseline解答\DeepSeaGLM-main\DeepSeaGLM-main\baseline\mountain_baseline\merged_files.txt


In [4]:
import pandas as pd
import os

def extract_status_values(directory):
    for filename in os.listdir(directory):
        filepath = os.path.join(directory, filename)
        # 过滤CSV和Excel文件
        if filename.lower().endswith(('.csv', '.xlsx')):
            print(f"\n文件名：{filename}")
            
            try:
                # 读取文件
                if filename.lower().endswith('.csv'):
                    df = pd.read_csv(filepath)
                else:
                    df = pd.read_excel(filepath, engine='openpyxl')
                
                # 检查status列是否存在
                if 'status' not in df.columns:
                    print("该文件没有status列")
                    continue
                
                # 提取唯一状态值并排除空值
                unique_status = df['status'].dropna().unique()
                
                # 转换并格式化输出
                if len(unique_status) > 0:
                    formatted_status = ' '.join(map(str, unique_status))
                    print(f"状态值：{formatted_status}")
                else:
                    print("status列无有效值")
                    
            except Exception as e:
                print(f"处理文件时发生错误：{str(e)}")

if __name__ == '__main__':
    import sys
    # 获取目录路径（通过命令行参数或手动输入）
    target_dir = input("请输入目录路径：")
    
    # 验证路径有效性
    if not os.path.isdir(target_dir):
        print("错误：提供的路径不是有效目录")
    else:
        extract_status_values(target_dir)



文件名：Ajia_plc_1.csv
状态值：False 关机 开机 A架摆出 缆绳挂妥 征服者出水 征服者起吊 征服者入水 缆绳解除 A架摆回 征服者落座 停电

文件名：device_13_11_meter_1311.csv
状态值：False 折臂吊车关机 折臂吊车开机 小艇检查完毕 小艇入水 小艇落座

文件名：device_13_14_meter_1314.csv
该文件没有status列

文件名：device_13_2_meter_1302.csv
该文件没有status列

文件名：device_13_3_meter_1303.csv
该文件没有status列

文件名：device_1_15_meter_115.csv
该文件没有status列

文件名：device_1_2_meter_102.csv
该文件没有status列

文件名：device_1_3_meter_103.csv
该文件没有status列

文件名：device_1_5_meter_105.csv
该文件没有status列

文件名：Jiaoche_plc_1.csv
该文件没有status列

文件名：Port1_ksbg_1.csv
该文件没有status列

文件名：Port1_ksbg_2.csv
该文件没有status列

文件名：Port1_ksbg_3.csv
该文件没有status列

文件名：Port1_ksbg_4.csv
该文件没有status列

文件名：Port1_ksbg_5.csv
该文件没有status列

文件名：Port2_ksbg_1.csv
该文件没有status列

文件名：Port2_ksbg_2.csv
该文件没有status列

文件名：Port2_ksbg_3.csv
该文件没有status列

文件名：Port2_ksbg_4.csv
该文件没有status列

文件名：Port3_ksbg_10.csv
该文件没有status列

文件名：Port3_ksbg_8.csv
该文件没有status列

文件名：Port3_ksbg_9.csv
状态值：False OFF_DP ON_DP

文件名：Port4_ksbg_7.csv
该文件没有status列

文件名：Port4_ksbg_8.csv
该文件没有statu

In [13]:
import pandas as pd
import os
from datetime import datetime

def parse_time_column(series):
    """增强型时间解析函数"""
    formats = [
        '%Y-%m-%d %H:%M:%S',    # 标准格式
        '%Y/%m/%d %H:%M:%S',    # 斜杠分隔
        '%Y%m%d%H%M%S',         # 紧凑格式
        '%d-%b-%y %H:%M',       # 日期-月份缩写-年份（01-OCT-23 12:30）
        '%m/%d/%Y %I:%M %p',    # 12小时制含AM/PM
        '%Y-%m-%dT%H:%M',       # ISO格式
        '%H:%M %Y-%m-%d',       # 时间在前
        '%Y%m%d_%H%M',          # 带下划线分隔
        '%Y-%m-%d',             # 仅日期
        '%H:%M:%S',             # 仅时间
        '%d.%m.%Y %H.%M'        # 点分隔格式
    ]
    
    # 多格式尝试解析
    for fmt in formats:
        try:
            return pd.to_datetime(series, format=fmt, errors='raise')
        except:
            continue
    
    # 最终尝试自动解析
    return pd.to_datetime(series, errors='coerce')

def find_hour_segments(hours):
    """生成连续小时时间段"""
    if not hours:
        return []
    
    # 转换numpy时间为pandas时间戳
    hours = [pd.Timestamp(h) for h in hours]
    
    segments = []
    start = current = hours[0]
    
    for h in hours[1:]:
        if h == current + pd.Timedelta(hours=1):
            current = h
        else:
            segments.append((start, current))
            start = current = h
    segments.append((start, current))
    return segments


def analyze_hour_ranges(directory):
    for filename in os.listdir(directory):
        filepath = os.path.join(directory, filename)
        if not filename.lower().endswith(('.csv', '.xlsx')):
            continue
            
        print(f"\n{'='*30}\n文件：{filename}")
        
        try:
            # 读取数据
            if filename.endswith('.csv'):
                df = pd.read_csv(filepath)
            else:
                df = pd.read_excel(filepath, engine='openpyxl')
            
            # 检查目标列
            if 'csvTime' not in df.columns:
                print("缺少 csvTime 列")
                continue
                
            # 转换时间列
            time_series = parse_time_column(df['csvTime'])
            valid_times = time_series.dropna()
            
            if valid_times.empty:
                print("没有有效时间数据")
                continue
                
            # 提取整点小时并去重排序
            valid_hours = valid_times.dt.floor('H')
            unique_hours = [pd.Timestamp(h) for h in valid_hours.sort_values().unique()]  # 关键修改
            
            # 生成时间段
            segments = find_hour_segments(unique_hours)
            
            # 格式化输出
            if not segments:
                print("未找到有效时间段")
                continue
                
            print(f"发现 {len(segments)} 个时间段：")
            for i, (start, end) in enumerate(segments, 1):
                start = pd.Timestamp(start)  # 类型保险
                end = pd.Timestamp(end)
                
                date_str = start.strftime("%Y-%m-%d")  # 新增日期
                start_time = start.strftime("%H:%M")
                end_time = end.strftime("%H:%M")
                
                if start == end:
                    print(f"[日期 {date_str}] 时段{i} {start_time}")
                else:
                    duration = (end - start).total_seconds() / 3600 + 1
                    print(f"[日期 {date_str}] 时段{i} {start_time} → {end_time} （持续{duration:.0f}小时）")
                    
        except Exception as e:
            print(f"处理异常：{str(e)}")


if __name__ == '__main__':
    import sys
    target_dir =  input("请输入目录路径：")
    
    if not os.path.exists(target_dir):
        print("路径不存在")
    else:
        analyze_hour_ranges(target_dir)



文件：Ajia_plc_1.csv
发现 93 个时间段：
[日期 2024-05-16] 时段1 16:00 → 20:00 （持续5小时）
[日期 2024-05-16] 时段2 22:00 → 00:00 （持续3小时）
[日期 2024-05-17] 时段3 02:00 → 08:00 （持续7小时）
[日期 2024-05-17] 时段4 10:00 → 14:00 （持续5小时）
[日期 2024-05-17] 时段5 17:00
[日期 2024-05-17] 时段6 19:00 → 20:00 （持续2小时）
[日期 2024-05-17] 时段7 22:00 → 02:00 （持续5小时）
[日期 2024-05-18] 时段8 04:00 → 06:00 （持续3小时）
[日期 2024-05-18] 时段9 08:00 → 09:00 （持续2小时）
[日期 2024-05-18] 时段10 11:00
[日期 2024-05-18] 时段11 14:00 → 19:00 （持续6小时）
[日期 2024-05-18] 时段12 21:00
[日期 2024-05-18] 时段13 23:00 → 00:00 （持续2小时）
[日期 2024-05-19] 时段14 02:00 → 09:00 （持续8小时）
[日期 2024-05-19] 时段15 11:00 → 19:00 （持续9小时）
[日期 2024-05-19] 时段16 21:00 → 05:00 （持续9小时）
[日期 2024-05-20] 时段17 07:00 → 10:00 （持续4小时）
[日期 2024-05-20] 时段18 12:00
[日期 2024-05-20] 时段19 17:00 → 20:00 （持续4小时）
[日期 2024-05-20] 时段20 22:00 → 02:00 （持续5小时）
[日期 2024-05-21] 时段21 04:00 → 05:00 （持续2小时）
[日期 2024-05-21] 时段22 09:00
[日期 2024-05-21] 时段23 11:00 → 13:00 （持续3小时）
[日期 2024-05-21] 时段24 15:00 → 16:00 （持续2小时）
[日期 2024-05-21] 时段25 18:00

验证新增api选择逻辑是否有用

In [1]:
import os
import sys
import json
from datetime import datetime
import pandas as pd
from ai_brain import get_answer

class AnswerVerifier:
    def __init__(self):
        # 测试用例
        self.test_cases = [
            {
                "id": "7",
                "question": "2024/8/23 A架开机的时间点（请以XX:XX输出）？",
                "expected_format": r"^\d{2}:\d{2}$"  # 期望格式：XX:XX
            },
            {
                "id": "8", 
                "question": "2024/8/24 征服者入水时间点（请以XX:XX输出）？",
                "expected_format": r"^\d{2}:\d{2}$"
            },
            {
                "id": "9",
                "question": "2024/8/24 揽绳解除的时间点（请以XX:XX输出）？",
                "expected_format": r"^\d{2}:\d{2}$"
            },
            {
                "id": "10",
                "question": "2024/8/24 A架摆回的时间点（请以XX:XX输出）？",
                "expected_format": r"^\d{2}:\d{2}$"
            },
            {
                "id": "11",
                "question": "2024/8/24 A架摆出的时间点（请以XX:XX输出）？",
                "expected_format": r"^\d{2}:\d{2}$"
            },
            {
                "id": "12",
                "question": "2024/8/24 8:45 ~ 8:55之间，发生了哪些关键动作？",
                "expected_format": r".*【.*】.*"  # 期望包含【】格式的动作
            },
            {
                "id": "13",
                "question": "2024/8/24 16:00 ~ 16:30之间，发生了哪些关键动作？",
                "expected_format": r".*【.*】.*"
            },
            {
                "id": "15",
                "question": "以征服者落座为标志，2024/8/23 的深海作业A作业结束的时间（请以XX:XX输出）？",
                "expected_format": r"^\d{2}:\d{2}$"
            },
            {
                "id": "19",
                "question": "2024/8/24 下午，A架开机发生在折臂吊车开机之前，是否正确？",
                "expected_format": r".*"  # 任意回答格式
            },
            {
                "id": "99",
                "question": "在2024年8月24日，小艇最后一次落座是什么时候（请以XX:XX输出）？",
                "expected_format": r"^\d{2}:\d{2}$"
            }
        ]
        
    def verify_answer_format(self, answer, expected_format):
        """验证答案格式是否符合预期"""
        import re
        return bool(re.match(expected_format, answer.strip()))

    def run_tests(self):
        """运行所有测试用例并记录结果"""
        results = []
        
        for case in self.test_cases:
            print(f"\n测试问题 {case['id']}:")
            print(f"问题: {case['question']}")
            
            try:
                # 获取答案
                answer = get_answer(case['question'])
                print(f"模型回答: {answer}")
                
                # 验证格式
                format_correct = self.verify_answer_format(answer, case['expected_format'])
                
                result = {
                    "id": case['id'],
                    "question": case['question'],
                    "answer": answer,
                    "format_correct": format_correct,
                    "expected_format": case['expected_format']
                }
                
                results.append(result)
                
                # 打印验证结果
                status = "✓" if format_correct else "✗"
                print(f"{status} 格式验证: {'通过' if format_correct else '失败'}")
                
            except Exception as e:
                print(f"Error processing question {case['id']}: {str(e)}")
                results.append({
                    "id": case['id'],
                    "question": case['question'],
                    "error": str(e)
                })
        
        return results

    def generate_report(self, results):
        """生成测试报告"""
        total = len(results)
        format_passed = sum(1 for r in results if r.get('format_correct', False))
        
        report = {
            "总测试数": total,
            "格式正确数": format_passed,
            "通过率": f"{(format_passed/total*100):.2f}%",
            "详细结果": results
        }
        
        # 保存报告
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        report_file = f'test_report_{timestamp}.json'
        
        with open(report_file, 'w', encoding='utf-8') as f:
            json.dump(report, f, ensure_ascii=False, indent=2)
            
        return report

def main():
    print("开始验证答案改进效果...")
    verifier = AnswerVerifier()
    
    print("\n运行测试用例...")
    results = verifier.run_tests()
    
    print("\n生成测试报告...")
    report = verifier.generate_report(results)
    
    print("\n测试总结:")
    print(f"总测试数: {report['总测试数']}")
    print(f"格式正确数: {report['格式正确数']}")
    print(f"通过率: {report['通过率']}")

if __name__ == "__main__":
    main()

所有文件夹均已存在。不再重新预处理数据。
需要预处理数据，请删除文件夹后重新运行。
开始验证答案改进效果...

运行测试用例...

测试问题 7:
问题: 2024/8/23 A架开机的时间点（请以XX:XX输出）？


D:\竞赛\数据挖掘竞赛\其他参赛者的答案\baseline解答\DeepSeaGLM-main\DeepSeaGLM-main\baseline\mountain_baseline\api.py:548: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  status_changes.loc[:, "csvTime"] = status_changes["csvTime"].dt.strftime(
D:\竞赛\数据挖掘竞赛\其他参赛者的答案\baseline解答\DeepSeaGLM-main\DeepSeaGLM-main\baseline\mountain_baseline\api.py:548: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  status_changes.loc[:, "csvTime"] = status_changes["csvTime"].dt.strftime(
D:\竞赛\数据挖掘竞赛\其他参赛者的答案\baseline解答\DeepSeaGLM-main\DeepSeaGLM-main\baseline\mountain_baseline\api.py:548: Futu

模型回答: 2024年8月23日A架的第一次开机时间点是08:03。
✗ 格式验证: 失败

测试问题 8:
问题: 2024/8/24 征服者入水时间点（请以XX:XX输出）？


D:\竞赛\数据挖掘竞赛\其他参赛者的答案\baseline解答\DeepSeaGLM-main\DeepSeaGLM-main\baseline\mountain_baseline\api.py:548: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  status_changes.loc[:, "csvTime"] = status_changes["csvTime"].dt.strftime(
D:\竞赛\数据挖掘竞赛\其他参赛者的答案\baseline解答\DeepSeaGLM-main\DeepSeaGLM-main\baseline\mountain_baseline\api.py:548: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  status_changes.loc[:, "csvTime"] = status_changes["csvTime"].dt.strftime(
D:\竞赛\数据挖掘竞赛\其他参赛者的答案\baseline解答\DeepSeaGLM-main\DeepSeaGLM-main\baseline\mountain_baseline\api.py:548: Futu

模型回答: 征服者入水的时间点是08:51。
✗ 格式验证: 失败

测试问题 9:
问题: 2024/8/24 揽绳解除的时间点（请以XX:XX输出）？
模型回答: 要确定2024年8月24日揽绳解除的具体时间点，我需要更多相关信息。例如，揽绳解除的背景是什么？有没有具体的时间安排或相关的通知？如果您能提供这些细节，我将能更准确地帮助您找到答案。如果没有具体信息，我只能猜测一个常见的时间点，如工作日的早上8点或9点等，但这并不准确。请提供更多细节以便我给出正确的答案。
✗ 格式验证: 失败

测试问题 10:
问题: 2024/8/24 A架摆回的时间点（请以XX:XX输出）？


...

模型回答: 在2024年8月24日8:45至8:55之间，A架发生了以下关键动作：
-08:49:08【征服者起吊】
-08:51:08【征服者入水】
-08:52:08【缆绳解除】
✗ 格式验证: 失败

测试问题 13:
问题: 2024/8/24 16:00 ~ 16:30之间，发生了哪些关键动作？


D:\竞赛\数据挖掘竞赛\其他参赛者的答案\baseline解答\DeepSeaGLM-main\DeepSeaGLM-main\baseline\mountain_baseline\api.py:548: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  status_changes.loc[:, "csvTime"] = status_changes["csvTime"].dt.strftime(
D:\竞赛\数据挖掘竞赛\其他参赛者的答案\baseline解答\DeepSeaGLM-main\DeepSeaGLM-main\baseline\mountain_baseline\api.py:548: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  status_changes.loc[:, "csvTime"] = status_changes["csvTime"].dt.strftime(


模型回答: 2024年8月24日16:00至16:30之间，发生的关键动作如下：

-A架：
-16:03:08开机
-16:25:08A架摆出

-折臂吊车：
-16:04:28折臂吊车开机
✗ 格式验证: 失败

测试问题 15:
问题: 以征服者落座为标志，2024/8/23 的深海作业A作业结束的时间（请以XX:XX输出）？
模型回答: 要确定2024年8月23日深海作业A的结束时间，我需要知道作业A的具体时长或开始时间。由于您提到“以征服者落座为标志”，这可能意味着作业的开始时间或某个关键事件的时间是已知的。请提供以下信息之一：

1.深海作业A的开始时间。
2.深海作业A的预计或实际持续时间。

有了这些信息，我才能准确计算出作业的结束时间。如果您不确定具体时间，请描述更多细节，以便我更好地帮助您。
✗ 格式验证: 失败

测试问题 19:
问题: 2024/8/24 下午，A架开机发生在折臂吊车开机之前，是否正确？
模型回答: 在查询的数据中，没有找到2024年8月24日下午（12:00至18:00期间）A架开机的记录。因此，无法直接确认A架开机是否发生在折臂吊车开机之前。如果需要进一步分析，请提供更多信息或调整查询条件。
✓ 格式验证: 通过

测试问题 99:
问题: 在2024年8月24日，小艇最后一次落座是什么时候（请以XX:XX输出）？


D:\竞赛\数据挖掘竞赛\其他参赛者的答案\baseline解答\DeepSeaGLM-main\DeepSeaGLM-main\baseline\mountain_baseline\api.py:548: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  status_changes.loc[:, "csvTime"] = status_changes["csvTime"].dt.strftime(
D:\竞赛\数据挖掘竞赛\其他参赛者的答案\baseline解答\DeepSeaGLM-main\DeepSeaGLM-main\baseline\mountain_baseline\api.py:548: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  status_changes.loc[:, "csvTime"] = status_changes["csvTime"].dt.strftime(
D:\竞赛\数据挖掘竞赛\其他参赛者的答案\baseline解答\DeepSeaGLM-main\DeepSeaGLM-main\baseline\mountain_baseline\api.py:548: Futu

模型回答: 2024年8月24日，小艇最后一次落座的时间是08:58。
✗ 格式验证: 失败

生成测试报告...

测试总结:
总测试数: 10
格式正确数: 1
通过率: 10.00%
